# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [21]:
import pandas as pd
import sqlite3
from IPython.display import display

conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",)
for table in tables: print(table[0],end=", ")
print("\b\b")

ALL TABLES: 
BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA, 


# REQUESTING USER'S ZIP TO COMPUTE PDP_REGION_CODE

In [2]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

Enter your zipcode: 07040


# YIELDING ALL PLANS THAT ARE IN THE REGION AND HAVE S-CONTRACTS

In [3]:
F_SUBSET = pd.read_sql("select * from PLAN_INFO\
                        where PDP_REGION_CODE is '"+str(user_pdp_region_code)+"'\
                        and CONTRACT_ID like 'S%'",conn)\
           [['CONTRACT_ID','PLAN_ID','SEGMENT_ID','FORMULARY_ID','PREMIUM','DEDUCTIBLE','PDP_REGION_CODE']]
F_SUBSET

,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,PDP_REGION_CODE
0,S0522,4,0,18041,30.4,405.0,4
1,S4802,78,0,18443,36.9,405.0,4
2,S4802,101,0,18439,79.5,0.0,4
3,S5601,8,0,18419,34.3,0.0,4
4,S5601,9,0,18420,84.6,0.0,4
5,S5617,18,0,18082,74.8,405.0,4
6,S5617,249,0,18083,52.9,0.0,4
7,S5660,106,0,18154,34.0,405.0,4
8,S5660,207,0,18155,91.7,350.0,4
9,S5660,220,0,18152,22.6,405.0,4


# REQUESTING DRUG NAMES AND STRENGTHS FROM USER

In [4]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):[]\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
user_generics

Enter how many drugs you wish to search: 3
Enter a generic drug name: metoprolol succinate
Enter a desired strength: 50
Enter a generic drug name: allopurinol
Enter a desired strength: 300
Enter a generic drug name: metformin
Enter a desired strength: 500


{('ALLOPURINOL', '300'): [],
 ('METFORMIN', '500'): [],
 ('METOPROLOL SUCCINATE', '50'): []}

# YIELDING TABLES FOR EACH DRUG WITH REQUIRED DATA FOR COMPUTING COST WITH BENEFICIARY_COST_FILE (LONG)

In [33]:
#WARNING: THIS TAKES A LONG-ISH TIME
for drug,strength in user_generics:
    user_generics[(drug,strength)] = pd.read_sql(
        " ".join(("select RXCUI_DATA.FULL_GENERIC_NAME, PLAN_INFO.FORMULARY_ID,\
                FORMULARY_DATA.RXCUI,FORMULARY_DATA.TIER_LEVEL_VALUE,\
                FORMULARY_DATA.PRIOR_AUTHORIZATION_YN,PLAN_INFO.CONTRACT_ID,\
                PLAN_INFO.PLAN_ID,PLAN_INFO.PREMIUM,PLAN_INFO.DEDUCTIBLE,\
                BENEFICIARY_COST.COVERAGE_LEVEL,BENEFICIARY_COST.DAYS_SUPPLY,\
                BENEFICIARY_COST.COST_AMT_PREF,BENEFICIARY_COST.COST_TYPE_PREF,\
                PLAN_INFO.PDP_REGION_CODE \
         from \
         FORMULARY_DATA join PLAN_INFO \
                        on substr(FORMULARY_DATA.FORMULARY_ID,4) = (PLAN_INFO.FORMULARY_ID) \
         join RXCUI_DATA on FORMULARY_DATA.RXCUI = RXCUI_DATA.RXCUI \
         join BENEFICIARY_COST on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                                  PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                                  cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = cast(BENEFICIARY_COST.TIER as integer) \
         where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
               PLAN_INFO.CONTRACT_ID like 'S%' and \
               upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and \
               RXCUI_DATA.STRENGTH like '"+strength+"%' and \
               (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
                cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
               BENEFICIARY_COST.DAYS_SUPPLY = 1 and \
               cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1").split()),
        conn)

In [34]:
for key in user_generics:
    display(user_generics[key])

,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE
0,24 HR metoprolol succinate 50 MG Extended Rele...,18039,866436,2,N,S5820,3,86.6,0.0,1.0,1,10.0,1.0,4
1,24 HR metoprolol succinate 50 MG Extended Rele...,18040,866436,1,N,S5921,349,33.2,405.0,1.0,1,1.0,1.0,4
2,24 HR metoprolol succinate 50 MG Extended Rele...,18041,866436,1,N,S0522,4,30.4,405.0,1.0,1,1.0,1.0,4
3,24 HR metoprolol succinate 50 MG Extended Rele...,18043,866436,1,N,S5921,386,26.8,405.0,0.0,1,0.0,1.0,4
4,24 HR metoprolol succinate 50 MG Extended Rele...,18043,866436,1,N,S5921,386,26.8,405.0,1.0,1,0.0,1.0,4
5,24 HR metoprolol succinate 50 MG Extended Rele...,18056,866436,2,N,S5810,38,34.9,325.0,0.0,1,2.0,1.0,4
6,24 HR metoprolol succinate 50 MG Extended Rele...,18056,866436,2,N,S5810,38,34.9,325.0,1.0,1,2.0,1.0,4
7,24 HR metoprolol succinate 50 MG Extended Rele...,18057,866436,2,N,S5810,278,19.7,405.0,0.0,1,3.0,1.0,4
8,24 HR metoprolol succinate 50 MG Extended Rele...,18057,866436,2,N,S5810,278,19.7,405.0,1.0,1,3.0,1.0,4
9,24 HR metoprolol succinate 50 MG Extended Rele...,18059,866436,2,N,S5768,127,56.2,0.0,1.0,1,2.0,1.0,4


,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE
0,Allopurinol 300 MG Oral Tablet,18039,197320,1,N,S5820,3,86.6,0.0,1.0,1,6.0,1.0,4
1,Allopurinol 300 MG Oral Tablet,18040,197320,1,N,S5921,349,33.2,405.0,1.0,1,1.0,1.0,4
2,Allopurinol 300 MG Oral Tablet,18041,197320,1,N,S0522,4,30.4,405.0,1.0,1,1.0,1.0,4
3,Allopurinol 300 MG Oral Tablet,18043,197320,1,N,S5921,386,26.8,405.0,0.0,1,0.0,1.0,4
4,Allopurinol 300 MG Oral Tablet,18043,197320,1,N,S5921,386,26.8,405.0,1.0,1,0.0,1.0,4
5,Allopurinol 300 MG Oral Tablet,18056,197320,1,N,S5810,38,34.9,325.0,0.0,1,1.0,1.0,4
6,Allopurinol 300 MG Oral Tablet,18056,197320,1,N,S5810,38,34.9,325.0,1.0,1,1.0,1.0,4
7,Allopurinol 300 MG Oral Tablet,18057,197320,1,N,S5810,278,19.7,405.0,0.0,1,0.0,1.0,4
8,Allopurinol 300 MG Oral Tablet,18057,197320,1,N,S5810,278,19.7,405.0,1.0,1,0.0,1.0,4
9,Allopurinol 300 MG Oral Tablet,18059,197320,1,N,S5768,127,56.2,0.0,1.0,1,1.0,1.0,4


,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE
0,24 HR Metformin hydrochloride 500 MG / saxagli...,18039,1043582,3,N,S5820,3,86.6,0.0,1.0,1,37.0,1.0,4
1,24 HR Metformin hydrochloride 500 MG / saxagli...,18040,1043582,3,N,S5921,349,33.2,405.0,1.0,1,30.0,1.0,4
2,24 HR Metformin hydrochloride 500 MG / saxagli...,18041,1043582,3,N,S0522,4,30.4,405.0,1.0,1,35.0,1.0,4
3,24 HR Metformin hydrochloride 500 MG / saxagli...,18130,1043582,3,N,S5993,1,47.7,405.0,1.0,1,34.0,1.0,4
4,24 HR Metformin hydrochloride 500 MG / saxagli...,18131,1043582,3,N,S5993,3,99.6,0.0,1.0,1,32.0,1.0,4
5,24 HR Metformin hydrochloride 500 MG / sitagli...,18039,1243848,3,N,S5820,3,86.6,0.0,1.0,1,37.0,1.0,4
6,24 HR Metformin hydrochloride 500 MG / sitagli...,18056,1243848,3,N,S5810,38,34.9,325.0,1.0,1,30.0,1.0,4
7,24 HR Metformin hydrochloride 500 MG / sitagli...,18057,1243848,3,N,S5810,278,19.7,405.0,1.0,1,47.0,1.0,4
8,24 HR Metformin hydrochloride 500 MG / sitagli...,18059,1243848,3,N,S5768,127,56.2,0.0,1.0,1,47.0,1.0,4
9,24 HR Metformin hydrochloride 500 MG / sitagli...,18082,1243848,3,N,S5617,18,74.8,405.0,1.0,1,30.0,1.0,4
